In [1]:
import random
from typing import TypedDict
import statistics


In [39]:
PET_LEVEL_TO_DROPRATE_MULTIPLIER = {
    1:1.0,
    2:1.0,
    3:1.0,
    4:1.33,
    5:1.67,
    6:2.0,
    7:2.33,
    8:2.67,
    9:3.0,
    10:3.33,
    11:3.67,
    12:4.0,
}

class PetDropRatesDict(TypedDict):
    item: str
    dropRate: int

In [77]:
dragon_pet_drops: PetDropRatesDict = {
    "Hoard":100,
    "Random Card":150,
    "Event Card":500
}

ada_dwarf_drops: PetDropRatesDict = {
    "Adamantium":200,
}


In [78]:
PET_LEVEL = 12

def run_simulation(sim_days: int, pet_drops_rates:PetDropRatesDict, pet_level:int):
    simulation_steps = sim_days*24
    pet_level_drop_rate_modifier = PET_LEVEL_TO_DROPRATE_MULTIPLIER[pet_level]
    
    for key in pet_drops_rates.keys():
        chance_to_drop = pet_level_drop_rate_modifier/pet_drops_rates.get(key)
        #print(f'Chance to drop {key} is {pet_level_drop_rate_modifier}/{pet_drops_rates.get(key)} = {pet_level_drop_rate_modifier/pet_drops_rates.get(key)} ')

    drop_results = dict() # stores list of total days since last drop per each item
    steps_since_last_drop = dict() # stores temporary days since last drop count for each item 
    for drop_item in pet_drops_rates.keys():
        drop_results[drop_item] = list()
        steps_since_last_drop[drop_item] = 0

    last_drop_step = 0
    for sim_step in range(simulation_steps):
        if((sim_step - last_drop_step) < 24): # one day delay per drop
            #print(f'Sim Step is  {sim_step}, last item drop is {last_drop_step} continuing!')
            steps_since_last_drop[drop_item] += 1
            continue

        for drop_item in pet_drops_rates.keys():
            chance_to_drop = pet_level_drop_rate_modifier/pet_drops_rates.get(drop_item)
            item_dropped = random.random() < chance_to_drop
            if(item_dropped):
                #print(f'Item {drop_item} dropped after {steps_since_last_drop[drop_item]} steps')
                drop_results[drop_item].append(steps_since_last_drop[drop_item]/24) #convert simulation hours to days
                steps_since_last_drop[drop_item] = 0
                last_drop_step = sim_step
            else:
                steps_since_last_drop[drop_item] += 1
    
    return drop_results
            

In [ ]:
NUM_SIMS = 10000
PET_DROPS = ada_dwarf_drops

SIM_YEARS = 10
PET_LEVEL = 12
all_results = dict()

for n in range(NUM_SIMS):
    for drop_item in PET_DROPS.keys():
        all_results[drop_item] = list()
    
    drop_results = run_simulation(SIM_YEARS*365, PET_DROPS, PET_LEVEL)
    for drop_item in PET_DROPS.keys():
        all_results[drop_item].extend(drop_results.get(drop_item))


for drop_item in PET_DROPS.keys():
    avg_drop_days = statistics.mean(all_results.get(drop_item))
    print(f'Avg days per drop of {drop_item} is {avg_drop_days}')